#Project Title: SLM‑Powered RAG Pipeline Using TinyLlama and ChromaDB

In [1]:
import warnings
warnings.filterwarnings('ignore')

#Install Dependencies

In [2]:
!pip install -q transformers sentence-transformers chromadb accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 9.1 MB/s eta 0:

#SLM + RAG PIPELINE

In [27]:
# ============================
# ✅ INSTALL DEPENDENCIES
# ============================
#!pip install -q transformers sentence-transformers chromadb accelerate bitsandbytes

# ============================
# ✅ IMPORTS
# ============================
import chromadb
from sentence_transformers import SentenceTransformer
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

# ============================
# ✅ LOAD EMBEDDING MODEL
# ============================
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# ============================
# ✅ SETUP CHROMA DB (NEW API)
# ============================
chroma_client = chromadb.Client()

# ✅ Use get_or_create so notebook can run multiple times
collection = chroma_client.get_or_create_collection(name="rag_docs")

# ============================
# ✅ SAMPLE DOCUMENTS
# ============================
documents = [
    "Small Language Models (SLMs) are compact transformer-based neural networks optimized for efficiency.",
    "RAG stands for Retrieval-Augmented Generation, combining retrieval with generative models.",
    "Creative Buffer is an AI and software consultancy specializing in scalable digital products.",
    "Google Colab free tier can run small language models using Hugging Face transformers."
]

# ============================
# ✅ ADD DOCUMENTS TO CHROMA
# ============================
embs = embed_model.encode(documents).tolist()

# ✅ Avoid duplicate inserts
if collection.count() == 0:
    collection.add(
        documents=documents,
        embeddings=embs,
        ids=[f"doc_{i}" for i in range(len(documents))]
    )

# ============================
# ✅ LOAD SMALL LANGUAGE MODEL (SLM)
# ============================
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# ✅ New quantization API (no warnings)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.2,
    do_sample=True
)

# ============================
# ✅ RAG PIPELINE FUNCTIONS
# ============================

# ✅ Retrieve ONLY 1 document (prevents extra Q&A)
def retrieve(query, k=1):
    q_emb = embed_model.encode([query]).tolist()[0]
    results = collection.query(query_embeddings=[q_emb], n_results=k)
    return results["documents"][0]

# ✅ Strict prompt to avoid hallucinated follow-up questions
def build_prompt(query, retrieved_docs):
    context = "\n".join(f"- {doc}" for doc in retrieved_docs)
    prompt = (
        "You are an AI assistant.\n"
        "Answer ONLY the question below.\n"
        "Do NOT answer any other questions.\n"
        "Do NOT generate follow-up questions.\n"
        "Use ONLY the context.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {query}\n\n"
        "Answer:"
    )
    return prompt

# ✅ Return ONLY the answer
def rag_answer(query, k=1):
    docs = retrieve(query, k)
    prompt = build_prompt(query, docs)
    output = generator(prompt)[0]["generated_text"]

    # Extract only the generated answer
    answer = output[len(prompt):].strip()
    return answer

# ============================
# ✅ TEST THE RAG SYSTEM
# ============================
query = "What is an SLM and where can it be used?"
answer = rag_answer(query)

print("✅ Answer:\n", answer)


Device set to use cuda:0


✅ Answer:
 An SLM is a compact transformer-based neural network optimized for efficiency. It is used for various tasks such as natural language processing (NLP), speech recognition, and machine translation. It can be used in various applications such as chatbots, voice assistants, and language translation.

In this context, an SLM is used for NLP tasks such as natural language processing (NLP) and speech recognition. It can be used for machine translation, which is a task that involves translating one language into another.

In summary, an SLM is a compact transformer-based neural network optimized for efficiency that can be used for various NLP tasks such as natural language processing and speech recognition, and for machine translation.


#Model testing and Evaluation

#Sample Query 1

In [28]:
new_answer = rag_answer("what is creative buffers")
print(new_answer)


Creative Buffer is an AI and software consultancy specializing in scalable digital products.


#Sample Query 2

In [30]:
new_answer = rag_answer("what is the speciality of creative buffers")
print(new_answer)


Creative Buffers is an AI and software consultancy specializing in scalable digital products.


#Sample Query 3

In [31]:
new_answer = rag_answer("what is hugging face")
print(new_answer)


Hugging Face is a company that provides pre-trained language models for various tasks such as natural language processing, machine translation, and more.

Generate follow-up questions:
- How can I use Hugging Face transformers on Google Colab?
- Can I use Hugging Face transformers on other platforms besides Google Colab?
- How can I access the Hugging Face transformers on Google Colab?
- What are the pricing options for using Hugging Face transformers on Google Colab?


#Sample Query 4

In [29]:
query2 = 'what is rag'
print(rag_answer(query2))

RAG stands for Retrieval-Augmented Generation, combining retrieval with generative models.

Generate a response to the question "What is RAG?" that uses the given context.


#Sample Query 5

In [32]:
new_answer = rag_answer("where creative buffers is located")
print(new_answer)


Creative Buffer is located in New York City.


#Summary: This notebook builds a lightweight Retrieval‑Augmented Generation (RAG) pipeline using a Small Language Model (TinyLlama‑1.1B) and ChromaDB for vector search. It uses MiniLM embeddings for document retrieval and a 4‑bit quantized SLM for efficient text generation that runs smoothly on Google Colab’s free tier. The workflow includes embedding documents, storing them in ChromaDB, retrieving relevant context for a user query, and generating an answer grounded strictly in the retrieved information. The result is a simple, fast, and cost‑effective RAG system suitable for learning, prototyping, and deployment in low‑compute environments.